# One more dataset that will teach you alot

- Main thing in this datset is that you will develop a habit of seeing confusion matrix since you can't trust on accuracy when data is imabalanced.

- You will see how easy is this to up-sample a data.

- Not much informative thing is done by me in this but its a good dataset to make your hands dirty.

In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm import tqdm 

In [ ]:
df = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#having a glance at multivariate analysis
plt.figure(figsize = (30,30))
sns.pairplot(df,hue = 'DEATH_EVENT')

In [ ]:
#Since its a medical data it will be useful to see imbalancement
df_positive = df[df['DEATH_EVENT'] == 1]
df_negative = df[df['DEATH_EVENT'] == 0]
print('percantage positive deaths are {}%'.format((df_positive.shape[0]/df.shape[0])*100))
print('percantage negative deaths are {}%'.format((df_negative.shape[0]/df.shape[0])*100))

In [ ]:
label = df['DEATH_EVENT']
df.drop('DEATH_EVENT',axis = 1,inplace = True)
col_names = df.columns
#scaling the features
scaler = StandardScaler()
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns = col_names)
scaled_df.head()

In [ ]:
#splitting the data
X_train,X_test,y_train,y_test = train_test_split(scaled_df, label,test_size = 0.3,random_state = 42)

#just seeing which neighbour will fit best
acc = []
for i in tqdm(range(1,50)):
    model = KNeighborsClassifier(n_neighbors = i)
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    acc.append(accuracy_score(y_test,pred))
plt.figure(figsize = (10,10))
plt.plot(range(1,50),acc, marker = 'o')
plt.xlabel('n_neighbor')
plt.ylabel('accuracy')
plt.title('accuracy graph')


In [ ]:
(acc)

In [ ]:
#Just predicting accuracy through cross-validation
model = KNeighborsClassifier(n_neighbors = 16)
cv_score = cross_val_score(model,scaled_df,label,cv = 10)
print(cv_score)
print(min(cv_score))
print(max(cv_score))
print(cv_score.mean())

In [ ]:
df.head()

In [ ]:
#seeing distribution for age
plt.figure(figsize=(15,5))
plt.hist(df['age'],bins = 50,edgecolor = 'black')
plt.xlabel('Age range')
plt.ylabel('Frequency')
plt.title('Age Distribution Graph')

In [ ]:
#seeing distribution for blood pressure
plt.figure(figsize=(15, 5))
plt.bar(df['high_blood_pressure'].unique(), df['high_blood_pressure'].value_counts())
plt.xlabel('NO/Yes high blood pressure')
plt.ylabel('Frequency')
plt.title('Graph for blood pressure')

In [ ]:
plt.figure(figsize=(15, 5))
plt.bar(df['smoking'].unique(), df['smoking'].value_counts())
plt.xlabel('NO/Yes smoking')
plt.ylabel('Frequency')
plt.title('Graph for Smoking')

In [ ]:
plt.figure(figsize=(15, 5))
plt.bar(df['sex'].unique(), df['sex'].value_counts())
plt.xlabel('female/male for sex')
plt.ylabel('Frequency')
plt.title('Graph for sex distribution')

In [ ]:
#checking confusion matrix
model.fit(X_train,y_train)
pred = model.predict(X_test)
cm = confusion_matrix(y_test,pred)
cm

false negative rate is high which is not good because it means that a person is having a disease but model is showing that he's not having which can be fatal for man this is due to imbalanced dataset Now we are going to upsample it.

In [ ]:
scaled_df.shape,label.shape

In [ ]:
#upsampling our dataset
from imblearn.over_sampling import RandomOverSampler
os = RandomOverSampler(random_state = 42)
X_train_res,y_train_res = os.fit_sample(scaled_df, label)
X_train_res.shape,y_train_res.shape

In [ ]:
model = KNeighborsClassifier(n_neighbors = 16)
cv_score = cross_val_score(model,X_train_res,y_train_res,cv = 10)
print(cv_score)
print(min(cv_score))
print(max(cv_score))
print(cv_score.mean())

In [ ]:
#checking outliers
plt.figure(figsize=(15,15))
for i,col in enumerate(['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_sodium','time']):
    plt.subplot(2,3,i+1)
    plt.boxplot(scaled_df[col])
    plt.title(col)

In [ ]:
#Removing outliers
per_25 = np.percentile(scaled_df['platelets'],25)
per_75 = np.percentile(scaled_df['platelets'],75)
IQR = (per_75 - per_25)*1.5
min_value = per_25 -IQR
max_value = per_75 + IQR
avg_value = scaled_df['platelets'].mean()
def treat_outliers(col):
    if col < min_value or col > max_value:
        col = avg_value
    return col
scaled_df['platelets'] = scaled_df['platelets'].apply(treat_outliers)

In [ ]:
plt.figure(figsize=(15,15))
for i,col in enumerate(['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_sodium','time']):
    plt.subplot(2,3,i+1)
    plt.boxplot(scaled_df[col])
    plt.title(col)

some fault happened in outlier treatment you have to identify it by using notebook titled remove outlier with one function.

In [ ]:
scaled_df.drop(['creatinine_phosphokinase','ejection_fraction','platelets'],axis = 1, inplace = True)
model = KNeighborsClassifier(n_neighbors = 6)
cv_score = cross_val_score(model,scaled_df,label,cv = 10)
print(cv_score)
print(min(cv_score))
print(max(cv_score))
print(cv_score.mean())

In [ ]:
#checking confusion matrix
X_train, X_test, y_train, y_test = train_test_split(X_train_res,y_train_res, test_size = 0.2)
model.fit(X_train, y_train)
pred = model.predict(X_test)
cm = confusion_matrix(y_test,pred)
cm

See the false negative decreased but the false positives increased it happens that if you will reduce one other will increase.